Germany is one of the countries with the largest number of people infected by the COVID-19 pandemic . 
At the moment the number of recently infected people is relatively low and the lockdown measures are being gradually lifted.
But won't it cause a new raise of the epidemy?

In this kernel we will take a look at the current situation in Germany in details.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from datetime import date, timedelta

from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Choosing time period:

In [ ]:
last_day = date(2020,5,8)
first_day = date(2020,2,25) 

In [ ]:
demographics = pd.read_csv('/kaggle/input/covid19-tracking-germany/demographics_de.csv')
#demographics.head(5)

In [ ]:
events = pd.read_csv('/kaggle/input/covid19-data-germany-robert-koch-institute/covid19_events_measures.csv')
#events.head()

In [ ]:
population = demographics[['state','population']].groupby('state').sum()

*Loading data from RKI (data is not complete for the last days, in this kernel it is used only for age and gender statistics) : *

In [ ]:
df_old = pd.read_csv('/kaggle/input/covid19-tracking-germany/covid_de.csv')

*Loading data from Morgenpost :*

In [ ]:
df = pd.read_csv('https://funkeinteraktiv.b-cdn.net/history.v4.csv')

df_ger = df[(df.label=='Deutschland')&(df.date>=int(first_day.strftime('%Y%m%d')))&(df.date<=int(last_day.strftime('%Y%m%d')))][
    ['label', 'date', 'confirmed', 'deaths', 'recovered']]
df_states = df[(df.label_parent=='Deutschland')&(df.date>=int(first_day.strftime('%Y%m%d')))&(df.date<=int(last_day.strftime('%Y%m%d')))][
    ['label', 'date', 'confirmed', 'deaths', 'recovered']]

# **Entire Germany**

In [ ]:
df_ger['active'] = df_ger['confirmed'] - df_ger['recovered']

In [ ]:
plt.figure(figsize=(12,6))
datelist = pd.date_range(end = last_day, periods = df_ger.shape[0]).tolist()
plt.plot(datelist, df_ger.confirmed, label='confirmed cases', color='b')
plt.plot(datelist, df_ger.recovered, label='recovered', color='g')
plt.plot(datelist, df_ger.active, label='active', color='orange')
plt.plot(datelist, df_ger.deaths, label='deaths', color='r')

plt.legend()
plt.title("Germany")
plt.grid()
plt.show()

**Moving (rolling) average**

Official data for COVID-19 in Germany have weekly oscillations (it can be seen well on some plots below).

This is why it is worth to smooth the values and calculate moving average for 7 days. 
So instead the value for particular day we will just take the mean of values for this day and the 6 days before.  

In [ ]:
df_ger["confirmed_ma"] = ""
df_ger["deaths_ma"] = ""
df_ger["recovered_ma"] = ""

for i in df_ger.index:
    df_ger.loc[i, "confirmed_ma"] = round(np.mean(df_ger.loc[i-6:i, "confirmed"]))
    df_ger.loc[i, "deaths_ma"] = round(np.mean(df_ger.loc[i-6:i, "deaths"]))
    df_ger.loc[i, "recovered_ma"] = round(np.mean(df_ger.loc[i-6:i, "recovered"]))
    
df_ger['active_ma'] = df_ger['confirmed_ma'] - df_ger['recovered_ma']

In [ ]:
plt.figure(figsize=(12,6))
#plt.yscale(value='log')
datelist = pd.date_range(end = date.today(), periods = df_ger.shape[0]).tolist()
plt.plot(datelist, df_ger.confirmed_ma, label='confirmed cases (m.a.)', color='b')
plt.plot(datelist, df_ger.recovered_ma, label='recovered (m.a.)', color='g')
plt.plot(datelist, df_ger.active_ma, label='active (m.a.)', color='orange')
plt.plot(datelist, df_ger.deaths_ma, label='deaths (m.a.)', color='r')

plt.legend()
plt.title("Germany: data with moving average")
plt.grid()
plt.show()

### **Daily new confirmed cases and deaths**

In [ ]:
df_ger['confirmed_delta'] = df_ger['confirmed'] - np.append([0], df_ger['confirmed'].values[:-1])
df_ger['deaths_delta'] = df_ger['deaths'] - np.append([0], df_ger['deaths'].values[:-1])

In [ ]:
df_ger["confirmed_delta_ma"] = ""
df_ger["deaths_delta_ma"] = ""

for i in df_ger.index:
    df_ger.loc[i, "confirmed_delta_ma"] = round(np.mean(df_ger.loc[i-6:i, "confirmed_delta"]))
    df_ger.loc[i, "deaths_delta_ma"] = round(np.mean(df_ger.loc[i-6:i, "deaths_delta"]))

In [ ]:
datelist = pd.date_range(end=last_day, periods=df_ger.shape[0]).tolist()
plt.figure(figsize=(15,7))

plt.plot(datelist, df_ger.confirmed_delta, label='confirmed daily', ls="--", color='b', lw=1)
plt.plot(datelist, df_ger.deaths_delta*10, label='deaths daily * 10', ls="--", color='r', lw=1)
plt.plot(datelist, df_ger.confirmed_delta_ma, label='confirmed daily (moving average)', color='b', lw=3)
plt.plot(datelist, df_ger.deaths_delta_ma*10, label='deaths_delta * 10 (moving average)', color='r', lw=3)
plt.legend()
plt.title("Germany - Daily New Cases and Deaths", fontsize=15)
plt.grid()


**Note**: on the plot above the number of deaths is multiplied by 10 for better visibility. 

In [ ]:
datelist = pd.date_range(end=last_day, periods=df_ger.shape[0]).tolist()
plt.figure(figsize=(15,7))

plt.yscale(value='log')
plt.plot(datelist, df_ger.confirmed_delta, label='confirmed daily', ls="--", color='b', lw=1)
plt.plot(datelist, df_ger.deaths_delta, label='deaths daily', ls="--", color='r', lw=1)
plt.plot(datelist, df_ger.confirmed_delta_ma, label='confirmed daily (moving average)', color='b', lw=3)
plt.plot(datelist, df_ger.deaths_delta_ma, label='deaths_delta (moving average)', color='r', lw=3)
plt.legend()
plt.title("Germany - Daily New Cases and Deaths (logarithmic scale)", fontsize=15)
plt.grid()

## **Growth Rate**

In [ ]:
datelist = pd.date_range(end = last_day, periods = df_ger.shape[0]).tolist()

fig, axs = plt.subplots(1, 2, figsize=(24, 7), sharey=True)
fig.suptitle("Germany - growth rate (moving average)", fontsize=24)

axs[0].plot(datelist[1:], df_ger.confirmed_delta_ma.values[1:]/df_ger.confirmed_delta_ma.values[:-1], label='confirmed growth rate (moving average)', color='b')
axs[0].plot(datelist[1:], [1]*(df_ger.shape[0]-1), color='g')
axs[0].legend(fontsize=17)
axs[0].set_title("Confirmed cases", fontsize=18)
axs[0].tick_params(axis='x', labelsize=15)
axs[0].tick_params(axis='y', labelsize=15)
axs[0].grid()

axs[1].plot(datelist[1:], df_ger.deaths_delta_ma.values[1:]/df_ger.deaths_delta_ma.values[:-1], label='deaths growth rate (moving average)', color='r')
axs[1].plot(datelist[1:], [1]*(df_ger.shape[0]-1), color='g')
axs[1].legend(fontsize=17)
axs[1].set_title("Deaths", fontsize=18)
axs[1].tick_params(axis='x', labelsize=15)
axs[1].tick_params(axis='y', labelsize=15)
axs[1].grid()

Here we can see, again, that the number of new confirmed cases was at the peak a month ago (intersection with the green line on the 3th-4th April) and now goes down. 
Deaths daily deltas had maximum in the time period between the 5th April and 25th April (approximately). 

Also we can note that the deaths growth rate is slightly higher in the last days than could be expected. Hopefully it's just some fluctuation and not a start of new growth.  

### **Daily increase of confirmed cases in % of the total number of confirmed cases at the moment**

In [ ]:
k=1  # won't show first k days
datelist = pd.date_range(end = date.today(), periods = df_ger.shape[0]-k).tolist()

plt.figure(figsize = (12,7))
plt.grid()
plt.plot(datelist, df_ger.confirmed_delta[k:] / df_ger.confirmed[k:] * 100)
plt.plot(datelist, df_ger.confirmed_delta[k:] / df_ger.active[k:] * 100, label='only active cases')
plt.legend()

plt.title("Confirmed cases - Daily increase (%)")
plt.show()

In [ ]:
# % delta to cumulative (moving average) (на сколько % увеличилось чиcло кейсов в этот день)

k=3 # exclude first k days
datelist = pd.date_range(end = last_day, periods = df_ger.shape[0]-k).tolist()

plt.figure(figsize = (12,7))
plt.grid()
plt.plot(datelist, df_ger.confirmed_delta_ma[k:] / df_ger.confirmed_ma[k:] * 100, label='all cases')
plt.plot(datelist, df_ger.confirmed_delta_ma[k:] / df_ger.active_ma[k:] * 100, label='only active cases')
plt.legend()

plt.title("Confirmed cases (moving average) - Daily increase (%)")
plt.show()

# **Federal states**

In [ ]:
states = df_states.label.unique()

df_states['confirmed_delta'] = ""
df_states['deaths_delta'] = ""

for s in states:
    df_states.loc[df_states.label==s, 'confirmed_delta'] = df_states.loc[df_states.label==s, 'confirmed'] \
            - np.append([0], df_states.loc[df_states.label==s,'confirmed'].values[:-1])
    df_states.loc[df_states.label==s, 'deaths_delta'] = df_states.loc[df_states.label==s, 'deaths'] \
            - np.append([0], df_states.loc[df_states.label==s,'deaths'].values[:-1])

In [ ]:
for s in states[:-1]:
    plt.figure(figsize=(12,5))
    datelist = pd.date_range(end = last_day, periods = df_states[df_states.label==s].shape[0]).tolist()
    #plt.plot(datelist, df_states[df_states.label==s].active, label='active')
    plt.plot(datelist, df_states[df_states.label==s].confirmed, color='b', label='conf')
    plt.plot(datelist, df_states[df_states.label==s].recovered, color='g', label='recovered')
    plt.plot(datelist, df_states[df_states.label==s].deaths, color='r', label='deaths')

    plt.title(s)
    plt.legend()
    plt.grid()

Unfortunately we can't rely on the recovered-data. Mostly it isn't the real data, but only estimated one (sometimes it's even obvious on plot - for example Hamburg).
That's why I will operate mostly with values of confirmed cases and deaths.

### **Federal states: Daily Cases and Deaths**

In [ ]:
df_states["confirmed_ma"] = ""
df_states["deaths_ma"] = ""

df_states["confirmed_delta_ma"] = ""
df_states["deaths_delta_ma"] = ""


for s in states:
    for i in df_states[df_states.label==s].index:
        df_states.loc[i, "confirmed_ma"] = round(np.mean(df_states[df_states.label==s].loc[i-6:i].confirmed))
        df_states.loc[i, "deaths_ma"] = round(np.mean(df_states[df_states.label==s].loc[i-6:i].deaths))
        
        df_states.loc[i, "confirmed_delta_ma"] = round(np.mean(df_states[df_states.label==s].loc[i-6:i].confirmed_delta))
        df_states.loc[i, "deaths_delta_ma"] = round(np.mean(df_states[df_states.label==s].loc[i-6:i].deaths_delta))

In [ ]:
for s in states[:-1]:
    
    fig, axs = plt.subplots(1, 2, figsize=(25, 7), sharey=True)
    
    datelist = pd.date_range(end = date.today()-timedelta(days=1), periods = df_states[df_states.label==s].shape[0]-1).tolist()
    axs[0].plot(datelist, df_states[df_states.label==s].confirmed_delta_ma[:-1], label='confirmed daily  (moving average)', lw=3)
    axs[0].plot(datelist, df_states[df_states.label==s].deaths_delta_ma[:-1]*10, label='deaths daily * 10  (moving average)', color='r')
    axs[0].legend(fontsize=15)
    axs[0].tick_params(axis='x', labelsize=15)
    axs[0].tick_params(axis='y', labelsize=15)
    axs[0].grid()
    
    datelist = pd.date_range(end = date.today()-timedelta(days=1), periods = df_states[df_states.label==s].shape[0]-1).tolist()
    axs[1].plot(datelist, df_states[df_states.label==s].confirmed_delta[:-1], label='confirmed daily', lw=3)
    axs[1].plot(datelist, df_states[df_states.label==s].deaths_delta[:-1]*10, label='deaths daily * 10', color='r')
    axs[1].legend(fontsize=15)
    axs[0].tick_params(axis='x', labelsize=15)
    axs[0].tick_params(axis='y', labelsize=15)
    axs[1].grid()
    
    fig.suptitle(s, fontsize=25)

**Note: the number of deaths is multiplied by 10 for better visibility.**

As we can see, plots of federal states differ. But generally (almost) all states have passed the peaks.

It is interesting: seems like Brandenburg, Thüringen and Bremen have even two peaks.
Moreover, the dynamic of the number of confirmed cases in Bremen in comparison with other states doesn't look good. It may still be growing.  

### **States: Daily Increase in %**

Ratio of the daily number of new confirmed cases to the total number of cases at that moment.

Only smoothed data; the original data is too "chaotical"

In [ ]:
k=18 # won't show first k days
datelist = pd.date_range(end = last_day, periods = df_ger.shape[0]-k).tolist()

plt.figure(figsize = (15,8))
plt.grid()
for s in states[:-1]:
    plt.plot(datelist, np.array(df_states[df_states.label==s].confirmed_delta_ma[k:]) / np.array(df_states[df_states.label==s].confirmed_ma[k-1:-1]) * 100, 
             label=s)
plt.legend()
plt.title("States: daily increase of confirmed cases (%)")
plt.show()

### **Federal states: Statistics per 100 000 population**

In [ ]:
# per 100 000 population

temp = []
temp_d = []
for s in states[:-1]:
    c = df_states.loc[(df_states.date==20200413)&(df_states.label==s), 'confirmed'].values[0]
    d = df_states.loc[(df_states.date==20200413)&(df_states.label==s), 'deaths'].values[0]
    
    if s=='Baden-Württemberg':
        p = population.loc['Baden-Wuerttemberg'].values[0]
    elif s=='Thüringen':
        p = population.loc['Thueringen'].values[0]
    else:
        p = population.loc[s].values[0]
        
    temp.append(c*100000/p)
    temp_d.append(d*100000/p)
    

In [ ]:
plt.figure(figsize=(12,8))
plt.grid(axis='x')
plt.title("Confirmed cases and fatalities per 100 000 population")
plt.barh(states[:-1], temp, label='confirmed')
plt.barh(states[:-1], temp_d, label='deaths')
plt.legend()


In [ ]:
df_states["pop"] = ""

for s in states[:-1]:
    if s=='Baden-Württemberg':
        p = population.loc['Baden-Wuerttemberg'].values[0]
    elif s=='Thüringen':
        p = population.loc['Thueringen'].values[0]
    else:
        p = population.loc[s].values[0]
        
    for i in df_states[df_states.label==s].index:
        df_states.loc[i, 'pop'] = p
        
df = df_states[df_states.label != "weitere Fälle bundesweit"]
df_states = df

df_states["conf_pro_pop"] = df_states["confirmed"] * 10000 / df_states["pop"] 
df_states["deaths_pro_pop"] = df_states["deaths"] * 10000 / df_states["pop"] 

In [ ]:
k=15  # won't show first k days
datelist = pd.date_range(end=last_day, periods=df_states[df_states.label=='Berlin'].shape[0]-k).to_list()

plt.figure(figsize=(15,8))
plt.title("History of the number of confirmed cases per 100 000 population")
for s in states[:-1]:
    plt.plot(datelist, df_states[df_states.label==s]["conf_pro_pop"][k:], label=s)
plt.legend()
plt.grid()

First, three states were the leaders (Hamburg, Baden-Würtenberg, Bayern), with Saarland joined a little bit later.
The dynamic in other states is more or less stable, except for Bremen, where the percent of positive tested population has been increasing faster in the last month. 

### **States: Lethality**

In [ ]:
plt.figure(figsize=(12,8))
plt.grid(axis='x')
plt.barh(states[:-1], np.array(temp_d)/np.array(temp)*100, label='letality, %')
plt.legend()

We can see that the states have very different statistics. 

So, Bremen has the third smallest number of confirmed cases per 100 000 people, but is far ahead in lethality(4.1%) (other testing strategy? situation in health care? outbreaks in nursing homes?) At the same time Berlin has about 3,5 times less the percent of died people(1.2%) and the medium number of confirmed cases per 100 000 people.

Let's illustrate the situation in every state in more detail. 

### **Federal states: Statistics (gender, age) :**

In [ ]:
stat_ger = df_old[['gender', 'age_group', 'cases', 'deaths']].groupby(['gender', 'age_group']).sum()
stat_ger = stat_ger.reset_index()

demo_ger = demographics[['gender', 'age_group', 'population']].groupby(['gender', 'age_group']).sum()
demo_ger = demo_ger.reset_index()

In [ ]:
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}%'.format(round(height*100,2)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom'
                    #, fontsize=14
                    )

The next plots show distribution of confirmed cases, deaths and lethality for different ages and gender (in absolute value and normalized to population with selected age and gender).

In [ ]:
x = np.arange(6)
ages = stat_ger.age_group.unique()
width=0.4
color1 = 'lightsteelblue'
color2 = 'tan'
legend_fontsize = 15
title_fontsize = 15
tick_fontsize = 14


fig = plt.figure(figsize=(15, 15))
gs = gridspec.GridSpec(3, 2)

ax = fig.add_subplot(gs[0, 0])
ax.bar(x - width/2, stat_ger[stat_ger.gender=="M"].cases, width, label='Men', color=color1)
ax.bar(x + width/2, stat_ger[stat_ger.gender=="F"].cases, width, label='Women', color=color2)
ax.set_title("Germany: cases in different ages/gender", fontsize=title_fontsize)
ax.set_xticks(x)
ax.set_xticklabels(ages)
ax.tick_params(axis='x', labelsize=tick_fontsize)
ax.tick_params(axis='y', labelsize=tick_fontsize)
ax.legend(fontsize=legend_fontsize)

ax = fig.add_subplot(gs[0, 1])
rects1 = ax.bar(x - width/2, np.array(stat_ger[stat_ger.gender=="M"].cases) / 
                np.array(demo_ger[demo_ger.gender=="male"].population), width, label='Men', color=color1)
rects2 = ax.bar(x + width/2, np.array(stat_ger[stat_ger.gender=="F"].cases) / 
                np.array(demo_ger[demo_ger.gender=="female"].population), width, label='Women', color=color2)
ax.set_title("Germany: cases (% of population in the group)", fontsize=title_fontsize)
ax.set_xticks(x)
ax.set_xticklabels(ages)
ax.tick_params(axis='x', labelsize=tick_fontsize)
ax.tick_params(axis='y', labelsize=tick_fontsize)
ax.legend(fontsize=legend_fontsize)
autolabel(rects1)
autolabel(rects2)

ax = fig.add_subplot(gs[1, 0])
ax.bar(x - width/2, stat_ger[stat_ger.gender=="M"].deaths, width, label='Men', color=color1)
ax.bar(x + width/2, stat_ger[stat_ger.gender=="F"].deaths, width, label='Women', color=color2)
ax.set_title("Germany: deaths", fontsize=title_fontsize)
ax.set_xticks(x)
ax.set_xticklabels(ages)
ax.tick_params(axis='x', labelsize=tick_fontsize)
ax.tick_params(axis='y', labelsize=tick_fontsize)
ax.legend(fontsize=legend_fontsize)

ax = fig.add_subplot(gs[1, 1])
rects3 = ax.bar(x - width/2, np.array(stat_ger[stat_ger.gender=="M"].deaths) / 
                np.array(demo_ger[demographics.gender=="male"].population), width, label='Men', color=color1)
rects4 = ax.bar(x + width/2, np.array(stat_ger[stat_ger.gender=="F"].deaths) / 
                np.array(demo_ger[demographics.gender=="female"].population), width, label='Women', color=color2)
ax.set_title("Germany: deaths (% of population in the group)", fontsize=title_fontsize)
ax.set_xticks(x)
ax.set_xticklabels(ages)
ax.tick_params(axis='x', labelsize=tick_fontsize)
ax.tick_params(axis='y', labelsize=tick_fontsize)
ax.legend(fontsize=legend_fontsize)
autolabel(rects3)
autolabel(rects4)

ax = fig.add_subplot(gs[2, 0])
rects5 = ax.bar(x - width/2, np.array(stat_ger[stat_ger.gender=="M"].deaths) / np.array(stat_ger[stat_ger.gender=="M"].cases), 
                width, label='Men', color=color1)
rects6 = ax.bar(x + width/2, np.array(stat_ger[stat_ger.gender=="F"].deaths) / np.array(stat_ger[stat_ger.gender=="F"].cases), 
                width, label='Women', color=color2)
ax.set_title("Germany: fatality rate", fontsize=title_fontsize)
ax.set_xticks(x)
ax.set_xticklabels(ages)
ax.tick_params(axis='x', labelsize=tick_fontsize)
ax.tick_params(axis='y', labelsize=tick_fontsize)
ax.legend(fontsize=legend_fontsize)
autolabel(rects5)
autolabel(rects6)

plt.tight_layout()
plt.show()

The same information for every federal state and comparison with the plot above:

In [ ]:
color1 = 'lightsteelblue'
color2 = 'tan'
legend_fontsize = 15
title_fontsize = 15
tick_fontsize = 14

for s in states[:-1]:
    if (s=="Baden-Württemberg"):
        s = "Baden-Wuerttemberg"
    if(s=="Thüringen"):
        s = "Thueringen"
    t = df_old[df_old.state==s][['gender', 'age_group', 'cases', 'deaths']].groupby(['gender', 'age_group']).sum()
    t = t.reset_index()

    x = np.arange(6)
    ages = t.age_group.unique()
    width=0.4

    fig = plt.figure(figsize=(15, 15))
    gs = gridspec.GridSpec(3, 2, figure=fig)

    ax = fig.add_subplot(gs[0, 0])
    ax.bar(x - width/2, t[t.gender=="M"].cases, width, label='Men')
    ax.bar(x + width/2, t[t.gender=="F"].cases, width, label='Women')
    ax.set_title(s + ": cases", fontsize=title_fontsize)
    ax.set_xticks(x)
    ax.set_xticklabels(ages)
    ax.legend(fontsize=legend_fontsize)
    ax.tick_params(axis='x', labelsize=tick_fontsize)
    ax.tick_params(axis='y', labelsize=tick_fontsize)

    ax = fig.add_subplot(gs[0, 1])
    rects1 = ax.bar(x - width/2, np.array(t[t.gender=="M"].cases) / 
                np.array(demographics[(demographics.state==s) & (demographics.gender=="male")].population), width, label='Men')
    rects2 = ax.bar(x + width/2, np.array(t[t.gender=="F"].cases) / 
                np.array(demographics[(demographics.state==s) & (demographics.gender=="female")].population), width, label='Women')
    rects1_0 = ax.bar(x - width/2, np.array(stat_ger[stat_ger.gender=="M"].cases) / 
                np.array(demo_ger[demo_ger.gender=="male"].population), width/2, fill=False, edgecolor=color1, label='Men (Germany total)')
    rects2_0 = ax.bar(x + width/2, np.array(stat_ger[stat_ger.gender=="F"].cases) / 
                np.array(demo_ger[demo_ger.gender=="female"].population), width/2, fill=False, edgecolor=color2, label='Women (Germany total)')
    ax.set_title(s + ": cases (% of population in the group)", fontsize=title_fontsize)
    ax.set_xticks(x)
    ax.set_xticklabels(ages)
    ax.legend(fontsize=legend_fontsize)
    ax.tick_params(axis='x', labelsize=tick_fontsize)
    ax.tick_params(axis='y', labelsize=tick_fontsize)
    autolabel(rects1)
    autolabel(rects2)

    ax = fig.add_subplot(gs[1, 0])
    ax.bar(x - width/2, t[t.gender=="M"].deaths, width, label='Men')
    ax.bar(x + width/2, t[t.gender=="F"].deaths, width, label='Women')
    ax.set_title(s + ": deaths", fontsize=title_fontsize)
    ax.set_xticks(x)
    ax.set_xticklabels(ages)
    ax.legend(fontsize=legend_fontsize)
    ax.tick_params(axis='x', labelsize=tick_fontsize)
    ax.tick_params(axis='y', labelsize=tick_fontsize)

    ax = fig.add_subplot(gs[1, 1])
    rects3 = ax.bar(x - width/2, np.array(t[t.gender=="M"].deaths) / 
                np.array(demographics[(demographics.state==s) & (demographics.gender=="male")].population), width, label='Men')
    rects4 = ax.bar(x + width/2, np.array(t[t.gender=="F"].deaths) / 
                np.array(demographics[(demographics.state==s) & (demographics.gender=="female")].population), width, label='Women')
    rects3_0 = ax.bar(x - width/2, np.array(stat_ger[stat_ger.gender=="M"].deaths) / 
                  np.array(demo_ger[demo_ger.gender=="male"].population), 
                  width/2, fill=False, edgecolor=color1, label='Men (Germany total)')
    rects4_0 = ax.bar(x + width/2, np.array(stat_ger[stat_ger.gender=="F"].deaths) / 
                  np.array(demo_ger[demo_ger.gender=="female"].population), 
                  width/2, fill=False, edgecolor=color2, label='Women (Germany total)')
    ax.set_title(s + ": deaths (% of population in the group)", fontsize=title_fontsize)
    ax.set_xticks(x)
    ax.set_xticklabels(ages)
    ax.legend(fontsize=legend_fontsize)
    ax.tick_params(axis='x', labelsize=tick_fontsize)
    ax.tick_params(axis='y', labelsize=tick_fontsize)
    autolabel(rects3)
    autolabel(rects4)

    ax = fig.add_subplot(gs[2, 0])
    rects5 = ax.bar(x - width/2, np.array(t[t.gender=="M"].deaths) / np.array(t[t.gender=="M"].cases), width, label='Men')
    rects6 = ax.bar(x + width/2, np.array(t[t.gender=="F"].deaths) / np.array(t[t.gender=="F"].cases), width, label='Women')
    rects5_0 = ax.bar(x - width/2, np.array(stat_ger[stat_ger.gender=="M"].deaths) / np.array(stat_ger[stat_ger.gender=="M"].cases), 
                  width/2, fill=False, edgecolor=color1, label='Men (Germany total)')
    rects6_0 = ax.bar(x + width/2, np.array(stat_ger[stat_ger.gender=="F"].deaths) / np.array(stat_ger[stat_ger.gender=="F"].cases), 
                  width/2, fill=False, edgecolor=color2, label='Women (Germany total)')
    ax.set_title(s + ": fatality rate", fontsize=title_fontsize)
    ax.set_xticks(x)
    ax.set_xticklabels(ages)
    ax.legend(fontsize=legend_fontsize)
    ax.tick_params(axis='x', labelsize=tick_fontsize)
    ax.tick_params(axis='y', labelsize=tick_fontsize)
    autolabel(rects5)
    autolabel(rects6)

    plt.tight_layout()  
    plt.show()

Gradual lockdown exit in Germany started on the 20th of April, but any significant changes for the worse have not been observed. 

There are small changes in the growth rate and in the dynamic in some federal states during the last days. But it's too early to say that the situation is getting worse. I suppose it will be much more clear during the next one or two weeks. Let's hope that the crisis is behind us.